In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import getpass
import json
import uuid

from kgforge.core import KnowledgeGraphForge

from inference_tools.utils import fetch_rules, check_premises, apply_rule, get_rule_parameters
from inference_tools.query.elastic_search import set_elastic_view

In [ ]:
ENDPOINT = "https://bbp.epfl.ch/nexus/v1"

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
# This could be replaced by any util kg-inference-api implements to handle user forge sessions
FORGE_SESSIONS = {}
def allocate_forge_session(org, project):
    if (org, project) not in FORGE_SESSIONS:
        session = KnowledgeGraphForge(
            "../../configs/new-forge-config.yaml",
            endpoint=ENDPOINT,
            token=TOKEN, 
            bucket=f"{org}/{project}")
        FORGE_SESSIONS[(org, project)] = session
    return FORGE_SESSIONS[(org, project)]

In [ ]:
rule_forge = allocate_forge_session("bbp", "inference-rules")

__NOTE__ we generate IDs for rules explicitly, so that the view that serves the rules can index the id's as well.

See the view `https://bbp.epfl.ch/neurosciencegraph/data/rule-view` in `bbp/inference-rules`.

# SEU Morph recommendation rules

## 1. Shape-based recommendation (neurite features + TMD-based representation)

Register a new rule

In [ ]:
rule_id = f"https://bbp.epfl.ch/neurosciencegraph/data/{uuid.uuid4()}"

In [ ]:
shape_seu_rule = {
    "id": rule_id,
    "type": "DataGeneralizationRule",
    "_schemaProject": "https://bbp.epfl.ch/nexus/v1/projects/bbp/inference-rules",
    "description": "Get closest morphologies according to neurite features and TMD-based vectorization of persistence diagrams",
    "name": "Shape-based morphology recommendation",
    "searchQuery": {
        "type": "SimilarityQuery",
        "hasParameter": [
            {
                "type": "uri",
                "description": "URI of the similarity search target entity.",
                "name": "TargetResourceParameter"
            },
            {
                "type": "list",
                "description": "List of URIs of similarity models to ignore.",
                "name": "IgnoreModelsParameter",
                "optional": True
            }
        ],
        "k": 20,
        "queryConfiguration": [
            {
                "boosted": True,
                "boostingView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/2ec2a253-30a7-4e24-b0fd-99436f4c2945",
                    "type": "ElasticSearchView"
                },
                "description": "Neurite feature-based embedding",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/d0c21fd5-cb9c-445c-b0a4-94847ba61f5a",
                    "type": "EmbeddingModel",
                    "hasSelector": {
                        "type": "FragmentSelector",
                        "conformsTo": "https://bluebrainnexus.io/docs/delta/api/resources-api.html#fetch",
                        "value": "?rev=9"
                    },
                    "org": "dke",
                    "project": "embedding-pipelines"
                },
                "org": "bbp-external",
                "project": "seu",
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/21e480a8-f955-4ff1-a98a-7a5fd3200a54",
                    "type": "ElasticSearchView"
                },
                "statisticsView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/aacfccfc-74f6-45c2-ae36-f505d93d117a",
                    "type": "ElasticSearchView"
                }
            },
            {
                "boosted": True,
                "boostingView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/20c96b9f-ec8e-4596-876b-7878f869f1e7",
                    "type": "ElasticSearchView"
                },
                "description": "Unscaled TMD-based embedding",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/43965be4-72f9-4901-9a95-d9ca13da8fb4",
                    "type": "EmbeddingModel",
                    "hasSelector": {
                        "type": "FragmentSelector",
                        "conformsTo": "https://bluebrainnexus.io/docs/delta/api/resources-api.html#fetch",
                        "value": "?rev=7"
                    },
                    "org": "dke",
                    "project": "embedding-pipelines"
                },
                "org": "bbp-external",
                "project": "seu",
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/12e3c699-135c-4c69-8ee7-f2b13f1376f9",
                    "type": "ElasticSearchView"
                },
                "statisticsView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/47a4edd9-7c96-4247-842f-50b8393a4aff",
                    "type": "ElasticSearchView"
                }
            }
        ],
        "searchTargetParameter": "TargetResourceParameter"
    },
    "targetResourceType": "NeuronMorphology"
}

In [ ]:
# rule_forge.register(rule_forge.from_json(shape_seu_rule))

Update the existing rule resource

In [ ]:
rule_id = "https://bbp.epfl.ch/neurosciencegraph/data/ac5885c8-bb70-4336-ae7f-3e1425356fe8"

In [ ]:
shape_seu_rule = rule_forge.retrieve(rule_id)
print(shape_seu_rule)

Make your updates to the rule

In [ ]:
#

In [ ]:
rule_forge.update(rule_forge.from_json(shape_seu_rule))

## 2. Location-based recommendation (brain region embedding + coordinates + axon/dendrite coprojections)

Register a new rule

In [ ]:
rule_id = f"https://bbp.epfl.ch/neurosciencegraph/data/{uuid.uuid4()}"

In [ ]:
loc_seu_rule = {
    "id": rule_id,
    "type": "DataGeneralizationRule",
    "_schemaProject": "https://bbp.epfl.ch/nexus/v1/projects/bbp/inference-rules",
    "description": "Get closest morphologies according to their location given by: coordinates, brain region embedding (ontology-based), axon/dendrite co-projection.",
    "name": "Location-based morphology recommendation",
    "searchQuery": {
        "type": "SimilarityQuery",
        "hasParameter": [
            {
                "type": "uri",
                "description": "URI of the similarity search target entity.",
                "name": "TargetResourceParameter"
            },
            {
                "type": "list",
                "description": "List of URIs of similarity models to ignore.",
                "name": "IgnoreModelsParameter",
                "optional": True
            }
        ],
        "k": 20,
        "queryConfiguration": [
            {
                "boosted": True,
                "boostingView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/0c65f539-042e-415f-b66b-a9f88b8a270d",
                    "type": "ElasticSearchView"
                },
                "description": "Coordinates-based embedding",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/608fab85-0cc9-4ff9-a4bd-4249589b5889",
                    "type": "EmbeddingModel",
                    "hasSelector": {
                        "type": "FragmentSelector",
                        "conformsTo": "https://bluebrainnexus.io/docs/delta/api/resources-api.html#fetch",
                        "value": "?rev=8"
                    },
                    "org": "dke",
                    "project": "embedding-pipelines"
                },
                "org": "bbp-external",
                "project": "seu",
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/269940bf-a7e1-44bf-a209-c83c650f2a3c",
                    "type": "ElasticSearchView"
                },
                "statisticsView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/bacd3821-9121-4190-84a7-e7fd7773479c",
                    "type": "ElasticSearchView"
                }
            },
            {
                "boosted": True,
                "boostingView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/4f37c088-34ea-47d3-b0bd-93994cdae8d2",
                    "type": "ElasticSearchView"
                },
                "description": "Brain region ontology-based embedding",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/1c4fcd2e-000f-437b-b65b-844ee211105a",
                    "type": "EmbeddingModel",
                    "hasSelector": {
                        "type": "FragmentSelector",
                        "conformsTo": "https://bluebrainnexus.io/docs/delta/api/resources-api.html#fetch",
                        "value": "?rev=5"
                    },
                    "org": "dke",
                    "project": "embedding-pipelines"
                },
                "org": "bbp-external",
                "project": "seu",
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/2c43c500-6ba2-4d1f-9a96-e80e480816c7",
                    "type": "ElasticSearchView"
                },
                "statisticsView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/55bd1cec-835d-4481-833d-feb526310666",
                    "type": "ElasticSearchView"
                }
            },
            {
                "boosted": True,
                "boostingView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/1e94a9c2-137c-433b-9559-1afd35eec98c",
                    "type": "ElasticSearchView"
                },
                "description": "Axon co-projection-based embedding",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/84519407-ad30-4d31-877e-1d6560325393",
                    "type": "EmbeddingModel",
                    "hasSelector": {
                        "type": "FragmentSelector",
                        "conformsTo": "https://bluebrainnexus.io/docs/delta/api/resources-api.html#fetch",
                        "value": "?rev=10"
                    },
                    "org": "dke",
                    "project": "embedding-pipelines"
                },
                "org": "bbp-external",
                "project": "seu",
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/a2cf748b-2f8f-4f6f-ad3f-0089e54e81a5",
                    "type": "ElasticSearchView"
                },
                "statisticsView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/9a98934f-ad05-4986-8b82-99352374da23",
                    "type": "ElasticSearchView"
                }
            },
            {
                "boosted": True,
                "boostingView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/800a6f6a-ed84-4721-b795-73bc6d43108b",
                    "type": "ElasticSearchView"
                },
                "description": "Dendrite co-projection-based embedding",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/9fe6873b-ef6a-41b5-854a-382bc1be9fff",
                    "type": "EmbeddingModel",
                    "hasSelector": {
                        "type": "FragmentSelector",
                        "conformsTo": "https://bluebrainnexus.io/docs/delta/api/resources-api.html#fetch",
                        "value": "?rev=10"
                    },
                    "org": "dke",
                    "project": "embedding-pipelines"
                },
                "org": "bbp-external",
                "project": "seu",
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/5cc7619e-d3ea-475b-a0f4-63cec28d4f9b",
                    "type": "ElasticSearchView"
                },
                "statisticsView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/e9a024bc-1fff-4d51-877c-26b9b6b1fd79",
                    "type": "ElasticSearchView"
                }
            }
        ],
        "searchTargetParameter": "TargetResourceParameter"
    },
    "targetResourceType": "NeuronMorphology"
}

In [ ]:
# rule_forge.register(rule_forge.from_json(loc_seu_rule))

Update the existing rule resource

In [ ]:
rule_id = "https://bbp.epfl.ch/neurosciencegraph/data/70e8e757-1834-420c-bcc1-37ea850ddfe3"

In [ ]:
loc_seu_rule = rule_forge.retrieve(rule_id)
print(loc_seu_rule)

Make your updates to the rule

In [ ]:
#

In [ ]:
rule_forge.update(rule_forge.from_json(loc_seu_rule))

## 3. All-aspects recommendation

Register a new rule

In [ ]:
rule_id = f"https://bbp.epfl.ch/neurosciencegraph/data/{uuid.uuid4()}"

In [ ]:
all_seu_rule = {
    "id": rule_id,
    "type": "DataGeneralizationRule",
    "_schemaProject": "https://bbp.epfl.ch/nexus/v1/projects/bbp/inference-rules",
    "description": "Get closest morphologies according to their location and shape",
    "name": "All-aspect morphology recommendation",
    "searchQuery": {
        "type": "SimilarityQuery",
        "hasParameter": [
            {
                "type": "uri",
                "description": "URI of the similarity search target entity.",
                "name": "TargetResourceParameter"
            },
            {
                "type": "list",
                "description": "List of URIs of similarity models to ignore.",
                "name": "IgnoreModelsParameter",
                "optional": True
            }
        ],
        "k": 20,
        "queryConfiguration": [
            {
                "boosted": True,
                "boostingView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/0c65f539-042e-415f-b66b-a9f88b8a270d",
                    "type": "ElasticSearchView"
                },
                "description": "Coordinates-based embedding",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/608fab85-0cc9-4ff9-a4bd-4249589b5889",
                    "type": "EmbeddingModel",
                    "hasSelector": {
                        "type": "FragmentSelector",
                        "conformsTo": "https://bluebrainnexus.io/docs/delta/api/resources-api.html#fetch",
                        "value": "?rev=8"
                    },
                    "org": "dke",
                    "project": "embedding-pipelines"
                },
                "org": "bbp-external",
                "project": "seu",
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/269940bf-a7e1-44bf-a209-c83c650f2a3c",
                    "type": "ElasticSearchView"
                },
                "statisticsView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/bacd3821-9121-4190-84a7-e7fd7773479c",
                    "type": "ElasticSearchView"
                }
            },
            {
                "boosted": True,
                "boostingView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/4f37c088-34ea-47d3-b0bd-93994cdae8d2",
                    "type": "ElasticSearchView"
                },
                "description": "Brain region ontology-based embedding",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/1c4fcd2e-000f-437b-b65b-844ee211105a",
                    "type": "EmbeddingModel",
                    "hasSelector": {
                        "type": "FragmentSelector",
                        "conformsTo": "https://bluebrainnexus.io/docs/delta/api/resources-api.html#fetch",
                        "value": "?rev=5"
                    },
                    "org": "dke",
                    "project": "embedding-pipelines"
                },
                "org": "bbp-external",
                "project": "seu",
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/2c43c500-6ba2-4d1f-9a96-e80e480816c7",
                    "type": "ElasticSearchView"
                },
                "statisticsView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/55bd1cec-835d-4481-833d-feb526310666",
                    "type": "ElasticSearchView"
                }
            },
            {
                "boosted": True,
                "boostingView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/1e94a9c2-137c-433b-9559-1afd35eec98c",
                    "type": "ElasticSearchView"
                },
                "description": "Axon co-projection-based embedding",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/84519407-ad30-4d31-877e-1d6560325393",
                    "type": "EmbeddingModel",
                    "hasSelector": {
                        "type": "FragmentSelector",
                        "conformsTo": "https://bluebrainnexus.io/docs/delta/api/resources-api.html#fetch",
                        "value": "?rev=10"
                    },
                    "org": "dke",
                    "project": "embedding-pipelines"
                },
                "org": "bbp-external",
                "project": "seu",
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/a2cf748b-2f8f-4f6f-ad3f-0089e54e81a5",
                    "type": "ElasticSearchView"
                },
                "statisticsView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/9a98934f-ad05-4986-8b82-99352374da23",
                    "type": "ElasticSearchView"
                }
            },
            {
                "boosted": True,
                "boostingView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/800a6f6a-ed84-4721-b795-73bc6d43108b",
                    "type": "ElasticSearchView"
                },
                "description": "Dendrite co-projection-based embedding",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/9fe6873b-ef6a-41b5-854a-382bc1be9fff",
                    "type": "EmbeddingModel",
                    "hasSelector": {
                        "type": "FragmentSelector",
                        "conformsTo": "https://bluebrainnexus.io/docs/delta/api/resources-api.html#fetch",
                        "value": "?rev=10"
                    },
                    "org": "dke",
                    "project": "embedding-pipelines"
                },
                "org": "bbp-external",
                "project": "seu",
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/5cc7619e-d3ea-475b-a0f4-63cec28d4f9b",
                    "type": "ElasticSearchView"
                },
                "statisticsView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/e9a024bc-1fff-4d51-877c-26b9b6b1fd79",
                    "type": "ElasticSearchView"
                }
            },
            {
                "boosted": True,
                "boostingView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/2ec2a253-30a7-4e24-b0fd-99436f4c2945",
                    "type": "ElasticSearchView"
                },
                "description": "Neurite feature-based embedding",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/d0c21fd5-cb9c-445c-b0a4-94847ba61f5a",
                    "type": "EmbeddingModel",
                    "hasSelector": {
                        "type": "FragmentSelector",
                        "conformsTo": "https://bluebrainnexus.io/docs/delta/api/resources-api.html#fetch",
                        "value": "?rev=9"
                    },
                    "org": "dke",
                    "project": "embedding-pipelines"
                },
                "org": "bbp-external",
                "project": "seu",
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/21e480a8-f955-4ff1-a98a-7a5fd3200a54",
                    "type": "ElasticSearchView"
                },
                "statisticsView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/aacfccfc-74f6-45c2-ae36-f505d93d117a",
                    "type": "ElasticSearchView"
                }
            },
            {
                "boosted": True,
                "boostingView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/20c96b9f-ec8e-4596-876b-7878f869f1e7",
                    "type": "ElasticSearchView"
                },
                "description": "Unscaled TMD-based embedding",
                "embeddingModel": {
                    "id": "https://bbp.epfl.ch/nexus/v1/resources/dke/embedding-pipelines/_/43965be4-72f9-4901-9a95-d9ca13da8fb4",
                    "type": "EmbeddingModel",
                    "hasSelector": {
                        "type": "FragmentSelector",
                        "conformsTo": "https://bluebrainnexus.io/docs/delta/api/resources-api.html#fetch",
                        "value": "?rev=7"
                    },
                    "org": "dke",
                    "project": "embedding-pipelines"
                },
                "org": "bbp-external",
                "project": "seu",
                "similarityView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/12e3c699-135c-4c69-8ee7-f2b13f1376f9",
                    "type": "ElasticSearchView"
                },
                "statisticsView": {
                    "id": "https://bbp.epfl.ch/neurosciencegraph/data/47a4edd9-7c96-4247-842f-50b8393a4aff",
                    "type": "ElasticSearchView"
                }
            }
        ],
        "searchTargetParameter": "TargetResourceParameter"
    },
    "targetResourceType": "NeuronMorphology"
}

In [ ]:
# rule_forge.register(rule_forge.from_json(loc_seu_rule))

Update the existing rule resource

In [ ]:
rule_id = "https://bbp.epfl.ch/neurosciencegraph/data/abb1949e-dc16-4719-b43b-ff88dabc4cb8"

In [ ]:
all_seu_rule = rule_forge.retrieve(rule_id)
print(all_seu_rule)

Make your updates to the rule

In [ ]:
#

In [ ]:
rule_forge.update(rule_forge.from_json(all_seu_rule))

# SSCX Trace recommendation

__TODO__: currently, the premise of the following rules checks if the input region is a part of isocortex, based on the __mouse__ AllenCCFv3 hierarchy, but the rules themselves operate on the data from __rat__. This needs to be addressed.

## 1. Simulation Trace generalization for pyramidal neurons

Register a new rule

In [ ]:
rule_id = f"https://bbp.epfl.ch/neurosciencegraph/data/{uuid.uuid4()}"

In [ ]:
sim_trace_rule = {
    "id": rule_id,
    "type": "DataGeneralizationRule",
    "_schemaProject": "https://bbp.epfl.ch/nexus/v1/projects/bbp/inference-rules",
    "description": "Collect smulation traces belonging to PCs from other cortical regions",
    "name": "SSCX Simulation Trace generalization for cortical PCs",
    "premise": [
        {
            "type": "SparqlPremise",
            "hasBody": """
                SELECT ?x
                WHERE {
                    <$BrainRegionQueryParameter> <http://schema.org/isPartOf>* <http://api.brain-map.org/api/v2/data/Structure/315> .
                }
            """,
            "hasParameter": {
                "type": "uri",
                "description": "URI of the query brain region",
                "name": "BrainRegionQueryParameter"
            },
            "queryConfiguration": {
                "org": "neurosciencegraph",
                "project": "datamodels"
            }
        },
        {
            "type": "SparqlPremise",
            "hasBody": """
                SELECT ?x
                WHERE {
                    <$MTypeQueryParameter> rdfs:subClassOf* <https://neuroshapes.org/PyramidalNeuron> .
                }
            """,
            "hasParameter": {
                "type": "uri",
                "description": "URI of the query MType",
                "name": "MTypeQueryParameter"
            },
            "queryConfiguration": {
                "org": "neurosciencegraph",
                "project": "datamodels"
            }
        }
    ],
    "searchQuery": {
        "type": "QueryPipe",
        "head": {
            "type": "SparqlQuery",
            "hasBody": """
                SELECT ?brainRegion
                WHERE {
                    ?brainRegion <http://schema.org/isPartOf>* <http://api.brain-map.org/api/v2/data/Structure/315> .
                }
            """,
            "queryConfiguration": {
                "org": "neurosciencegraph",
                "project": "datamodels"
            },
            "resultParameterMapping": {
                "parameterName": "acceptableBrainRegions",
                "path": "brainRegion"
            }
        },
        "rest": {
            "type": "QueryPipe",
            "head": {
                "type": "SparqlQuery",
                "hasBody": """
                    SELECT ?mtype
                    WHERE {
                        ?mtype rdfs:subClassOf* <https://neuroshapes.org/PyramidalNeuron> .
                    }
                """,
                "queryConfiguration": {
                    "org": "neurosciencegraph",
                    "project": "datamodels"
                },
                "resultParameterMapping": {
                    "parameterName": "acceptableMTypes",
                    "path": "mtype"
                }
            },
            "rest": {
                "type": "ElasticSearchQuery",
                "hasBody": """
                    {
                        "query": {
                            "bool": {
                              "must": [
                                {
                                  "terms": {
                                    "@type": ["SingleCellSimulationTrace"]
                                  }
                                },
                                {
                                  "term": {
                                      "_deprecated": false
                                    }
                                },
                                {
                                  "nested": {
                                    "path": "annotation.hasBody",
                                    "query": {
                                      "terms": {
                                          "annotation.hasBody.@id": [$acceptableMTypes]
                                        }
                                    }
                                  }
                                }
                              ]
                            }
                        }
                    }
                """,
                "hasParameter": [
                    {
                        "type": "uri_list",
                        "description": "List of acceptable mtypes",
                        "name": "acceptableMTypes"
                    },
                    {
                        "type": "uri_list",
                        "description": "List of acceptable brain regions",
                        "name": "acceptableBrainRegions"
                    }
                ],
                "queryConfiguration": {
                    "elasticSearchView": {
                        "id": "https://bluebrain.github.io/nexus/vocabulary/sscx-simulation-trace-view"
                    },
                    "org": "public",
                    "project": "sscx"
                }
            }
        }
    },
    "targetResourceType": "Trace"
}

In [ ]:
# rule_forge.register(rule_forge.from_json(sim_trace_rule))

Update the existing rule resource.

In [ ]:
rule_id = "https://bbp.epfl.ch/neurosciencegraph/data/da2c61f5-59d9-4347-9c3a-0cdf2c093547"

In [ ]:
sim_trace_rule = rule_forge.retrieve(rule_id)
print(sim_trace_rule)

Make your updates to the rule

In [ ]:
#

In [ ]:
rule_forge.update(rule_forge.from_json(sim_trace_rule))

## 2. Trace generalization for pyramidal neurons

In [ ]:
rule_id = f"https://bbp.epfl.ch/neurosciencegraph/data/{uuid.uuid4()}"

In [ ]:
trace_rule = {
    "id": rule_id,
    "type": "DataGeneralizationRule",
    "_schemaProject": "https://bbp.epfl.ch/nexus/v1/projects/bbp/inference-rules",
    "description": "Collect traces belonging to PCs from SSCX regions",
    "name": "SSCX Trace generalization for cortical PCs",
    "premise": [
         {
            "type": "SparqlPremise",
            "hasBody": """
                  SELECT ?region
                  WHERE {
                      ?region <http://schema.org/isPartOf>* <http://api.brain-map.org/api/v2/data/Structure/315>.
                      VALUES (?region) {
                          $BrainRegionQueryParameter
                      }
                  }
            """,
            "hasParameter": {
                "type": "sparql_value_uri_list",
                "description": "List of URIs of the query brain regions",
                "name": "BrainRegionQueryParameter"
            },
            "queryConfiguration": {
                "org": "neurosciencegraph",
                "project": "datamodels"
            }
        },
        {
            "type": "SparqlPremise",
            "hasBody": """
                  SELECT ?x
                  WHERE {
                      ?mtype rdfs:subClassOf* <https://neuroshapes.org/PyramidalNeuron> .
                      VALUES (?mtype) {
                          $MTypeQueryParameter
                      }
                  }
            """,
            "hasParameter": {
                "type": "sparql_value_uri_list",
                "description": "URI of the query MTypes",
                "name": "MTypeQueryParameter"
            },
            "queryConfiguration": {
                "org": "neurosciencegraph",
                "project": "datamodels"
            }
        }
    ],
    "searchQuery": {
        "type": "QueryPipe",
        "head": {
            "type": "SparqlQuery",
            "hasBody": """
                SELECT ?mtype
                WHERE {
                    ?mtype rdfs:subClassOf* <https://neuroshapes.org/PyramidalNeuron> .
                }
            """,
            "queryConfiguration": {
                "org": "neurosciencegraph",
                "project": "datamodels"
            },
            "resultParameterMapping": {
                "parameterName": "acceptableMTypes",
                "path": "mtype"
            }
        },
        "rest": {
            "type": "QueryPipe",
            "head": {
                "type": "ElasticSearchQuery",
                "hasBody": """
                  {
                      "query": {
                        "bool": {
                          "must": [
                            {
                              "term": {
                                  "_deprecated": false
                                }
                            },
                            {
                              "nested": {
                                "path": "derivation.entity",
                                "query": {
                                  "exists": {
                                      "field": "derivation.entity.@id"
                                  }
                                }
                              }
                            },
                            {
                              "nested": {
                                "path": "annotation.hasBody",
                                "query": {
                                  "terms": {
                                      "annotation.hasBody.@id": [$acceptableMTypes]
                                    }
                                }
                              }
                            }
                          ]
                        }
                      }
                    }
                """,
                "hasParameter": {
                    "type": "uri_list",
                    "description": "List of acceptable mtypes",
                    "name": "acceptableMTypes"
                },
                "queryConfiguration": {
                    "elasticSearchView": {
                        "id": "https://bluebrain.github.io/nexus/vocabulary/sscx-morph-view"
                    },
                    "org": "public",
                    "project": "sscx"
                },
                "resultParameterMapping": {
                    "parameterName": "acceptablePatchedCells",
                    "path": "derivation.entity.@id"
                }
            },
            "rest": {
                "type": "ElasticSearchQuery",
                "hasBody": """
                    {
                      "query": {
                        "bool": {
                          "must": [
                            {
                              "term": {
                                  "_deprecated": false
                                }
                            },
                            {
                              "nested": {
                                "path": "derivation.entity",
                                "query": {
                                  "terms": {
                                      "derivation.entity.@id": [$acceptablePatchedCells]
                                    }
                                }
                              }
                            }
                          ],
                          "must_not": [
                              {
                              "nested": {
                                "path": "isPartOf",
                                "query": {
                                  "exists": {
                                      "field": "isPartOf.@id"
                                  }
                                }
                              }
                            }
                          ]
                        }
                      }
                    }
                """,
                "hasParameter": {
                    "type": "uri_list",
                    "description": "List of acceptable patched cells",
                    "name": "acceptablePatchedCells"
                },
                "queryConfiguration": {
                    "elasticSearchView": {
                        "id": "https://bluebrain.github.io/nexus/vocabulary/sscx-trace-view"
                    },
                    "org": "public",
                    "project": "sscx"
                }
            }
        }
    },
    "targetResourceType": "Trace"
}

In [ ]:
rule_forge.register(rule_forge.from_json(trace_rule))

Update the existing rule resource

In [ ]:
rule_id = "https://bbp.epfl.ch/neurosciencegraph/data/g0e761f5-59d9-4347-9c3a-0cdf2c104658"

In [ ]:
trace_rule = rule_forge.retrieve(rule_id)
print(trace_rule)

Make your updates to the rule

In [ ]:
#

In [ ]:
rule_forge.update(rule_forge.from_json(sim_trace_rule))